In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Google sheet - Get handle on it

A Google sheet is used to store the names of image files as well as the text transcribed from handwriting.

In [83]:
!pip install --upgrade -q gspread

In [84]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('fieldbook_pages')
worksheet_pages = sh.worksheet(title="pages")
worksheet_contrasted_pages = sh.worksheet(title="contrasted_pages")

# Azure Congnitive Services. 
+ Authenticate and Connect
+ For each link in the spreadsheet, transcribe and store in sheet

In [85]:
# Get keys to connect
import os
import pandas as pd

directory_config = "/content/drive/MyDrive/azure_config/" # Change this to suit your machine

path_config = os.path.join(directory_config,"cv.json")

# Verify the configuration file exists
if(os.path.exists(path_config)):
    print("Success.",path_config, "exists.")
else:
    print("Failure.",path_config, "does not exist.")

# Read the JSON file into a DataFrame
df_config = pd.read_json(path_config)
#print(df_config['COMPUTER_VISION_SUBSCRIPTION_KEY'].iloc[0])
#print(df_config['COMPUTER_VISION_ENDPOINT'].iloc[0])

# Store as enivonmental variables
os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY'] = df_config['COMPUTER_VISION_SUBSCRIPTION_KEY'].iloc[0]
os.environ['COMPUTER_VISION_ENDPOINT'] = (df_config['COMPUTER_VISION_ENDPOINT'].iloc[0])

# Do some basic validation
if len(os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']) == 32:
    print("Success, COMPUTER_VISION_SUBSCRIPTION_KEY is loaded.")
else:
    print("Error, The COMPUTER_VISION_SUBSCRIPTION_KEY is not the expected length, please check it.")

Success. /content/drive/MyDrive/azure_config/cv.json exists.
Success, COMPUTER_VISION_SUBSCRIPTION_KEY is loaded.


In [86]:
# Install what is required to connect to Azure Cognitive Services Computer Vision
# Run this once on your machine. If you are using Google Colab, run this once per session.
!pip install --upgrade azure-cognitiveservices-vision-computervision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached azure_cognitiveservices_vision_computervision-0.9.0-py2.py3-none-any.whl (39 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
  Using cached msrest-0.7.1-py3-none-any.whl (85 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached azure_core-1.25.0-py3-none-any.whl (178 kB)


In [88]:
# Run this once per session 

# Import the required libraries
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
import sys

# Get your Computer Vision subscription key from your environment variable.
if 'COMPUTER_VISION_SUBSCRIPTION_KEY' in os.environ:
    subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
else:      
    print("\nSet the COMPUTER_VISION_SUBSCRIPTION_KEY environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()

# Get your Computer Vision endpoint from your environment variable.
if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    endpoint = os.environ['COMPUTER_VISION_ENDPOINT']
else:
    print("\nSet the COMPUTER_VISION_ENDPOINT environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()

# Authenticate with Azure Cognitive Services.
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))


In [89]:
# Read just one file
import time
# This section is taken directly from:
# https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/ComputerVision/ComputerVisionQuickstart.py

# <snippet_read_call>
print("===== Read File - remote =====")
# Get an image with text. Set the url of the image to transcribe.
read_image_url = "https://pi.lib.uchicago.edu/1001/org/ochre/c14ddaa8-c833-42cc-be0b-a89624f1302c&load"

# Call API with URL and raw response (allows you to get the operation location). Call Azure using computervision_client with the URL.
read_response = computervision_client.read(read_image_url,  raw=True, model_version="2022-01-30-preview", language = "fr")
# </snippet_read_call>
  
# <snippet_read_response>
# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
        time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()

# </snippet_read_response>

===== Read File - remote =====
1 Juin.
[125.0, 30.0, 210.0, 30.0, 211.0, 59.0, 126.0, 58.0]
C'st un terrain situe a l' Est on la Renti ; inst contyn
[67.0, 85.0, 644.0, 89.0, 644.0, 122.0, 67.0, 117.0]
Le pek voyia à & maism ro'sine a fm muriau
[132.0, 340.0, 637.0, 340.0, 637.0, 371.0, 132.0, 370.0]
À la rent , pos ihr con Na Ih tivain - Jun 10m x3m
[64.0, 473.0, 628.0, 469.0, 628.0, 500.0, 64.0, 503.0]



In [90]:
# function to trascribe a link
import time

def get_text_from_image(read_image_url):

    # Call API with URL and raw response (allows you to get the operation location). Call Azure using computervision_client with the URL.
    read_response = computervision_client.read(read_image_url,  raw=True, model_version="2022-01-30-preview", language = "fr")  

    # Get the operation location (URL with an ID at the end) from the response
    read_operation_location = read_response.headers["Operation-Location"]
    # Grab the ID from the URL
    operation_id = read_operation_location.split("/")[-1]

    # Call the "GET" API and wait for it to retrieve the results 
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
            time.sleep(1)

    # Print the detected text, line by line
    return_value = ""
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                return_value = return_value + str(line.text) + "\n"                
    return(return_value)

In [ ]:
# Transcribe all rows in the sheet
row = 2
link ="----"
while link != None:
    link = worksheet_pages.cell(row, 4).value
    print(row,link)
    text_from_page = get_text_from_image(link)
    worksheet_pages.update_cell(row, 5, "'"+text_from_page)
    row = row + 1

# Transcribe images stored on Google Drive.
The images on the website are written in pencil and are difficult to read due to the low contrast between the color of pencil lead and paper. These images were processed to enhance the contrast by rendering them into black and white images.

Original:

https://ochre.lib.uchicago.edu/ochre?uuid=c14ddaa8-c833-42cc-be0b-a89624f1302c&load

Processed:

https://jeffblackadar.ca/fieldbook_antioch/ANT_FB_1932-003-0000.jpg

In [ ]:
# list images in a directory
import os 
for x in os.listdir('/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12'):
    if x.endswith(".jpg"):
        # Prints only text file present in My Folder
        print(x)

In [ ]:
# convert gray images to black and white
import cv2 
from google.colab.patches import cv2_imshow

import os

raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/'
contrast_images_dir = os.path.join(raw_images_dir,"contrasted/") 
for raw_image_file_name in os.listdir(raw_images_dir):
    if raw_image_file_name.endswith(".jpg"):
        print(raw_image_file_name)
        # Prints only text file present in My Folder

        raw_img = cv2.imread(os.path.join(raw_images_dir, raw_image_file_name))
        gray_image = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
        #https://pyimagesearch.com/2021/04/28/opencv-thresholding-cv2-threshold/
        #blurred_gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0) -- didn't really work here
        (thresh, binary) = cv2.threshold(gray_image, 208, 255, cv2.THRESH_BINARY)
        #cv2_imshow(img)
        cv2_imshow(gray_image)
        cv2.imwrite(os.path.join(contrast_images_dir,raw_image_file_name), binary)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [91]:
import cv2 
import numpy as np 

img =   cv2.imread("/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/ANT_FB_1932-003-0000.jpg")[...,0]
#bgray = cv2.imread(fname)[...,0]

blured1 = cv2.medianBlur(img,3)
blured2 = cv2.medianBlur(img,51)
divided = np.ma.divide(blured1, blured2).data
normed = np.uint8(255*divided/divided.max())
th, threshed = cv2.threshold(normed, 100, 255, cv2.THRESH_OTSU)

dst = np.vstack((img, blured1, blured2, normed, threshed)) 
cv2_imshow(dst)

NameError: ignored

In [ ]:
# function to trascribe an image stored on a drive
import time

def get_text_from_image(read_image_path):

    # Open the image
    read_image = open(read_image_path, "rb")

    # Call API with image and raw response (allows you to get the operation location). Call Azure using computervision_client with the image.
    read_response = computervision_client.read_in_stream(read_image, raw=True)

    # Get the operation location (URL with ID as last appendage)
    read_operation_location = read_response.headers["Operation-Location"]

    # Take the ID off and use to get results
    operation_id = read_operation_location.split("/")[-1]

    # Call the "GET" API and wait for the retrieval of the results

    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status.lower () not in ['notstarted', 'running']:
            break
            print ('Waiting for result...')
            time.sleep(10)

    # Print results, line by line
    return_value = ""
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                return_value = return_value + str(line.text) + "\n"  
                print(line.text)
                print(line.bounding_box)
    return(return_value)

In [ ]:
import os

row = 2
contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/'
for contrasted_image_file_name in os.listdir(contrast_images_dir):
    if contrasted_image_file_name.endswith(".jpg"):
        # Prints only text file present in My Folder
        print(contrasted_image_file_name)
        worksheet_contrasted_pages.update_cell(row, 1, contrasted_image_file_name[7:11])
        worksheet_contrasted_pages.update_cell(row, 3, contrasted_image_file_name[7:-4])
        
        text_on_image = get_text_from_image(os.path.join(contrast_images_dir,contrasted_image_file_name))
        worksheet_contrasted_pages.update_cell(row, 5, text_on_image)

        row = row + 1
        print(row,contrasted_image_file_name[7:-4])

# AWS

https://docs.aws.amazon.com/textract/latest/dg/analyzing-document-expense.html

code used from

https://github.com/Raman-Raje/colab_notebooks/blob/master/aws_textract/aws_textract.ipynb
Thanks to : https://github.com/Raman-Raje

In [ ]:
!pip install boto3
!pip install trp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
import re
import os
import io
import math
import boto3
import pandas as pd

aws_region = "ca-central-1"

In [ ]:
print('Enter your_access_key_id:')
aws_access_key_id = input()
print('Enter your_secret_access_key:')
aws_secret_access_key = input()
print("DELETE THIS OUTPUT!!")

In [ ]:
#Analyzes text in a document stored in an S3 bucket. Display polygon box around text and angled text 
import boto3
import io
from PIL import Image, ImageDraw
import cv2

def ShowBoundingBox(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],outline=boxColor)   

def ShowSelectedElement(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],fill=boxColor)  

# Displays information about a block returned by text detection and text analysis
def DisplayBlockInformation(block):
    print('Id: {}'.format(block['Id']))
    if 'Text' in block:
        print('    Detected: ' + block['Text'])
    print('    Type: ' + block['BlockType'])
   
    if 'Confidence' in block:
        print('    Confidence: ' + "{:.2f}".format(block['Confidence']) + "%")

    if block['BlockType'] == 'CELL':
        print("    Cell information")
        print("        Column:" + str(block['ColumnIndex']))
        print("        Row:" + str(block['RowIndex']))
        print("        Column Span:" + str(block['ColumnSpan']))
        print("        RowSpan:" + str(block['ColumnSpan']))    
    
    if 'Relationships' in block:
        print('    Relationships: {}'.format(block['Relationships']))
    print('    Geometry: ')
    print('        Bounding Box: {}'.format(block['Geometry']['BoundingBox']))
    print('        Polygon: {}'.format(block['Geometry']['Polygon']))
    
    if block['BlockType'] == "KEY_VALUE_SET":
        print ('    Entity Type: ' + block['EntityTypes'][0])
    
    if block['BlockType'] == 'SELECTION_ELEMENT':
        print('    Selection element detected: ', end='')

        if block['SelectionStatus'] =='SELECTED':
            print('Selected')
        else:
            print('Not selected')    
    
    if 'Page' in block:
        print('Page: ' + block['Page'])
    print()

def process_text_analysis(region, image_fp):
    """
    #Get the document from S3
    s3_connection = boto3.resource('s3')
                          
    s3_object = s3_connection.Object(bucket,document)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)

    # Analyze the document
    client = boto3.client('textract', region_name=region)
    
    image_binary = stream.getvalue()
    response = client.analyze_document(Document={'Bytes': image_binary},
        FeatureTypes=["TABLES", "FORMS"])
    """
    ### Uncomment to process using S3 object ###
    #response = client.analyze_document(
    #    Document={'S3Object': {'Bucket': bucket, 'Name': document}},
    #    FeatureTypes=["TABLES", "FORMS"])

    ### Uncomment to analyze a local file ###
    client = boto3.client('textract', region_name=region,aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
    with open(image_fp, 'rb') as img_file:
        ### To display image using PIL ###
        #image = Image.open()
        ### Read bytes ###
        img_bytes = img_file.read()
        response = client.analyze_document(Document={'Bytes': img_bytes}, FeatureTypes=["TABLES", "FORMS"])
    image = Image.open(image_fp)
    #Get the text blocks
    blocks=response['Blocks']
    width, height = image.size
    

    print ('Detected Document Text')
   
    # Create image showing bounding box/polygon the detected lines/text
    text_detected = ""
    for block in blocks:
        #DisplayBlockInformation(block)    
        draw=ImageDraw.Draw(image)
        if 'Text' in block:
            text_detected = text_detected + " "+ block['Text']
        # Draw bounding boxes for different detected response objects
        if block['BlockType'] == "KEY_VALUE_SET":
            if block['EntityTypes'][0] == "KEY":
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'red')
            else:
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'green')             
        if block['BlockType'] == 'TABLE':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'blue')
        if block['BlockType'] == 'CELL':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'yellow')
        if block['BlockType'] == 'SELECTION_ELEMENT':
            if block['SelectionStatus'] =='SELECTED':
                ShowSelectedElement(draw, block['Geometry']['BoundingBox'],width,height, 'blue')    
            
    # Display the image
    image.show()
    image.save('fred.png')
    return len(blocks), text_detected



#block_count, text_detected =process_text_analysis(aws_region)
#print("Blocks detected: " + str(block_count), text_detected)
    


In [ ]:
row = 2
contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/'
for row_counter in range(2,24):
    file_num = worksheet_contrasted_pages.get_values("C"+str(row_counter))[0][0]
    print(file_num)
    #ANT_FB_1932-003-0016.jpg
    contrasted_image_file_name = "ANT_FB_" + file_num + ".jpg"
    block_count, text_on_image =process_text_analysis(aws_region, os.path.join(contrast_images_dir,contrasted_image_file_name))
    worksheet_contrasted_pages.update_cell(row_counter, 6, text_on_image)

1932-003-0000
Detected Document Text
1932-003-0001
Detected Document Text
1932-003-0002
Detected Document Text
1932-003-0003
Detected Document Text
1932-003-0004
Detected Document Text
1932-003-0005
Detected Document Text
1932-003-0006
Detected Document Text
1932-003-0007
Detected Document Text
1932-003-0008
Detected Document Text
1932-003-0009
Detected Document Text
1932-003-0010
Detected Document Text
1932-003-0011
Detected Document Text
1932-003-0012
Detected Document Text
1932-003-0013
Detected Document Text
1932-003-0014
Detected Document Text
1932-003-0015
Detected Document Text
1932-003-0016
Detected Document Text
1932-003-0017
Detected Document Text
1932-003-0018
Detected Document Text
1932-003-0019
Detected Document Text
1932-003-0020
Detected Document Text
1932-003-0021
Detected Document Text


In [ ]:
# Write the data in the spreadsheet out as html files so it is more browsable.
# Like this:
# https://jeffblackadar.ca/fieldbook_antioch/2_ANT_FB_1932-003-0000.html

import os
row = 2

contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/'
for row_counter in range(2,24):
    file_num = worksheet_contrasted_pages.get_values("C"+str(row_counter))[0][0]
    print(file_num)
    #ANT_FB_1932-003-0016.jpg
    contrasted_image_file_name = "ANT_FB_" + file_num + ".jpg"
    print(contrasted_image_file_name)
    html_file = open(os.path.join("/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/html/", (str(row_counter)+"_"+contrasted_image_file_name[:-4]+".html")), "w")
    html_file.write('<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<title></title>\n</head>\n<body>\n<table>\n<tr>\n')
    html_file.write('<td>\n')
    html_file.write('<img src="'+contrasted_image_file_name+'">\n<br>\n')
    html_file.write('</td><td>\n')    
    html_file.write('<h2>Azure Cognitive Services</h2>\n<p>\n')
    azure_text = worksheet_contrasted_pages.get_values("E"+str(row_counter))
    for v in azure_text:
        for vs in v:  
            html_file.write(str(vs)+"<br>\n")
    html_file.write('</p>\n')
    html_file.write('<h2>AWS Textract</h2>\n<p>\n')
    aws_text = worksheet_contrasted_pages.get_values("F"+str(row_counter))
    for v in aws_text:
        for vs in v:  
            html_file.write(str(vs)+"<br>\n")
    html_file.write('</p>\n')
    
    print(row,contrasted_image_file_name[7:-4], azure_text, aws_text)
    if row_counter<23:
        next_file_num = worksheet_contrasted_pages.get_values("C"+str(row_counter+1))[0][0]
        next_file_name = str(row_counter+1)+"_"+"ANT_FB_" + next_file_num + ".html"
        html_file.write('Next: <a href="' + next_file_name + '">' + next_file_name + '</a>\n')
    html_file.write('</td>\n')
    html_file.write("</tr>\n</table>\n</body>\n</html>")
    html_file.close()

1932-003-0000
ANT_FB_1932-003-0000.jpg
2 1932-003-0000 [["1 Juin.\nC'est un terrain situé à l'Est or la Route ; il est conlyn\nl'un 1 derrières massimo ; plante de quelpas figuiers - dal\nacting but jeers . it off presseurs ; in ya reme'de mai's.\nPlusieurs tuns voision, d'environ 5m de profondeur, as servi\nÀ carrier ; men a extrait is press dailles, with fragment\ngardent belle apparence. Les travaux net avéli pas 11. Pust\nDe puits voisin A la maison voisine . for nouveau\nSean : - 11mbo . I st use seemite love to fouls .\nNone offirms & droit as faul un povagy, è prosimale\nÀ la conte , pi du coin NO in terrain - sun 10m xIm\nCommencement 1 travaux, 4 por, à Ifhe\n, Om40\n2. Jun\nA frame Divers monacies , une bagno . A foto ries orals\n4.\nremission ( patch fragments) profmeus au son : 200\n"]] [[' ) Juin. C\'st uin tanain situi a 1 Est is la Ronte ilst conlyn 1. Fun & burious menime; plants & guelpus togmen me anthoms but jurns it it preneure m y c remi\'d mais Plusen two valian, 

In [ ]:
# Read just one file --- cell may be garbage now
import time

# Change for AWS

# <snippet_read_call>
print("===== Read File - remote =====")
# Get an image with text. Set the url of the image to transcribe.
read_image_url = "https://pi.lib.uchicago.edu/1001/org/ochre/c14ddaa8-c833-42cc-be0b-a89624f1302c&load"

# Call API with URL and raw response (allows you to get the operation location). Call Azure using computervision_client with the URL.
#read_response = computervision_client.read(read_image_url,  raw=True, model_version="2022-01-30-preview", language = "fr")
# </snippet_read_call>
  
# <snippet_read_response>
# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
        time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()

# </snippet_read_response>